# ETH HMA 策略分析 - 4h级别深度分析

本notebook专注于4h级别的HMA策略分析，包括：
- 趋势识别和拐点分析
- 做多/做空策略表现
- 风险收益分析
- 策略优化建议

## 分析目标
- 基于HMA斜率变化的趋势识别
- 上涨趋势做多策略分析
- 下跌趋势做空策略分析
- 风险控制和收益优化


In [ ]:
# 导入必要的库和模块
import sys
import os
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# 设置中文字体支持
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei', 'SimHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.size'] = 12

# 设置Jupyter内联显示
%matplotlib inline

# 导入项目模块
from analyzers.trend_analyzer import TrendAnalyzer
from visualizers.jupyter_visualizer import JupyterVisualizer

print("✅ 所有模块导入完成")


In [ ]:
# 初始化分析器和可视化器
trend_analyzer = TrendAnalyzer()
jupyter_viz = JupyterVisualizer(figsize=(15, 10))

# 加载4h数据（假设已从上一个notebook加载）
if 'df_4h' in globals():
    print("✅ 4h数据已加载")
    print(f"📊 数据形状: {df_4h.shape}")
    print(f"📅 时间范围: {df_4h.index[0]} 到 {df_4h.index[-1]}")
else:
    print("❌ 请先运行数据加载notebook")
    # 备用数据加载
    data_file = Path("../src/utils/data/ETHUSDT_4h_processed_20250926_214833.parquet")
    if data_file.exists():
        df_4h = pd.read_parquet(data_file)
        print("✅ 备用数据加载成功")
    else:
        print("❌ 数据文件不存在")


In [ ]:
# 执行趋势分析
print("🔍 开始执行趋势分析...")

# 运行完整的趋势分析
analysis_result = trend_analyzer.run_complete_analysis(
    df_4h, 
    hma_period=45, 
    slope_threshold=0.001
)

print("✅ 趋势分析完成")
print(f"📊 分析结果包含: {list(analysis_result.keys())}")

# 提取关键结果
intervals = analysis_result.get('intervals', [])
events = analysis_result.get('events', [])
report = analysis_result.get('report', {})

print(f"📈 识别到 {len(intervals)} 个趋势区间")
print(f"🎯 识别到 {len(events)} 个事件")


In [ ]:
# 显示策略分析摘要
uptrend_analysis = report.get('uptrend_analysis', {})
downtrend_analysis = report.get('downtrend_analysis', {})

print("📊 4h级别策略分析摘要")
print("=" * 60)

# 做多策略摘要
print("\n🟢 做多策略 (上涨趋势):")
print(f"  总趋势数: {uptrend_analysis.get('total_uptrends', 0)}")
print(f"  平均理想收益: {uptrend_analysis.get('avg_long_ideal_profit', 0):.2f}%")
print(f"  平均实际收益: {uptrend_analysis.get('avg_long_actual_profit', 0):.2f}%")
print(f"  平均风险损失: {uptrend_analysis.get('avg_long_risk_loss', 0):.2f}%")
print(f"  平均风险收益比: {uptrend_analysis.get('avg_risk_reward_ratio', 0):.2f}")
print(f"  胜率: {uptrend_analysis.get('win_rate', 0):.2%}")

# 做空策略摘要
print("\n🔴 做空策略 (下跌趋势):")
print(f"  总趋势数: {downtrend_analysis.get('total_downtrends', 0)}")
print(f"  平均理想收益: {downtrend_analysis.get('avg_short_ideal_profit', 0):.2f}%")
print(f"  平均实际收益: {downtrend_analysis.get('avg_short_actual_profit', 0):.2f}%")
print(f"  平均风险损失: {downtrend_analysis.get('avg_short_risk_loss', 0):.2f}%")
print(f"  平均风险收益比: {downtrend_analysis.get('avg_risk_reward_ratio', 0):.2f}")
print(f"  胜率: {downtrend_analysis.get('win_rate', 0):.2%}")

print("\n" + "=" * 60)


In [ ]:
# 1. 价格走势和HMA曲线图
print("📈 生成价格走势和HMA曲线图...")

fig, ax = jupyter_viz.plot_price_and_hma(
    df_4h, 
    hma_col='HMA_45', 
    title="ETH 4h级别价格走势与HMA曲线",
    show_turning_points=True
)


In [ ]:
# 2. 策略表现分析图
print("📊 生成策略表现分析图...")

fig, axes = jupyter_viz.plot_strategy_performance(
    uptrend_analysis, 
    downtrend_analysis, 
    title="4h级别策略表现分析"
)


In [ ]:
# 3. 风险分析图
print("⚠️ 生成风险分析图...")

fig, axes = jupyter_viz.plot_risk_analysis(
    uptrend_analysis, 
    downtrend_analysis, 
    title="4h级别风险分析"
)


In [ ]:
# 4. 趋势区间分析图
print("📈 生成趋势区间分析图...")

fig, ax = jupyter_viz.plot_trend_intervals(
    df_4h, 
    intervals, 
    title="4h级别趋势区间分析"
)


In [ ]:
# 5. 综合仪表板
print("📊 生成综合仪表板...")

fig, axes = jupyter_viz.create_interactive_dashboard(
    df_4h, 
    uptrend_analysis, 
    downtrend_analysis
)


In [ ]:
# 6. 详细数据分析
print("🔍 详细数据分析...")

# 显示趋势区间详情
if intervals:
    print(f"\n📈 趋势区间详情 (前10个):")
    for i, interval in enumerate(intervals[:10]):
        print(f"  区间 {i+1}: {interval['direction']}趋势, "
              f"持续时间: {interval['duration']}周期, "
              f"价格变化: {interval['price_change_pct']:.2f}%, "
              f"PFE: {interval['pfe_pct']:.2f}%, "
              f"MAE: {interval['mae_pct']:.2f}%")

# 显示事件详情
if events:
    print(f"\n🎯 事件详情 (前10个):")
    for i, event in enumerate(events[:10]):
        print(f"  事件 {i+1}: {event['event_type']}, "
              f"时间: {event['timestamp']}, "
              f"价格: ${event['price']:.2f}, "
              f"影响: {event['impact']:.2f}%")

print(f"\n✅ 4h级别策略分析完成！")
